In [1]:
%load_ext autoreload
%autoreload 2
import xarray as xr
import matplotlib.pyplot as plt
import pandas as pd
import processing_netcdf as pcdf
import geopandas as gpd
from geopandas import GeoDataFrame
import shapely.geometry 
import numpy as np
from shapely import geometry as gmty
from geofeather import to_geofeather, from_geofeather
import glob
import os
import pyarrow
from xclim import ensembles as ens
from xclim import subset
import matplotlib.pyplot as plt

In [2]:
folder = "/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/"

# Select variable

In [3]:
var = "solidprcptot"

In [4]:
#Variable 
variable85="rcp85_"+var+"_seasonal"
variable45="rcp45_"+var+"_seasonal"
#variable="rcp[48]5_tg_mean_annual"

files85 = glob.glob(folder+"*"+variable85+".nc")
files45 = glob.glob(folder+"*"+variable45+".nc")
#ex: ACCESS1-3_rcp45_tn_mean_annual.nc
files85
files45

['/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/ACCESS1-3_rcp45_solidprcptot_seasonal.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/BNU-ESM_rcp45_solidprcptot_seasonal.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/CanESM2_rcp45_solidprcptot_seasonal.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/CMCC-CMS_rcp45_solidprcptot_seasonal.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/GFDL-ESM2M_rcp45_solidprcptot_seasonal.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/HadGEM2-CC_rcp45_solidprcptot_seasonal.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/INM-CM4_rcp45_solidprcptot_seasonal.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/IPSL-CM5A-LR_rcp45_solidprcptot_seasonal.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/IPSL-CM5B-LR_rcp45_solidprcptot_seasonal.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/MPI-ESM-LR_rcp45_solidprcptot_seasonal.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-

# FOR RCP 45

## Subsetting arrays by periods of time with Xclim

In [5]:
dsEns45= ens.create_ensemble(files45)
dsEns45.time.dt.season[0:4]
# Seasonal example 30 y means
listds = []
i=0
for s in dsEns45.time.dt.season[0:4]:
    print (i)
    tmp1 = dsEns45.sel(time=(dsEns45.time.dt.year>=1981))
    tmp1 = tmp1.sel(time=(tmp1.time.dt.season==s))
    #print(tmp1)
    i = i+1
    listds.append(tmp1)

0
1
2
3


## Loop to create all 4 dataframes

In [6]:
i = 0
df45list = []
for ds in listds:
    print ("Dataset number: ", i, "--------------")
    perc45 = ens.ensemble_percentiles(ds)
    print ("Percentiles Obteined")
    df45 = perc45.to_dataframe()
    df452 = df45.reset_index()
    df453 = df452.loc[df452["realization"] == 0].dropna()
    print ("Converted to DF")
    year_groups = {y:0 for y in range(1980,2011)}
    year_groups.update({y:1 for y in range(2041,2071)})
    year_groups.update({y:2 for y in range(2071,2101)})
    dfp = df453.groupby([df453.time.dt.year.map(year_groups), "lat","lon", "realization"]).mean()
    dfp452 =  dfp.reset_index()
    print ("Split in 3 different time periods")
    df45C = dfp452.copy()
    df45C[var+"_p10"] = round(df45C[var+"_p10"]/10,2)
    df45C[var+"_p50"] = round(df45C[var+"_p50"]/10,2)
    df45C[var+"_p90"] = round(df45C[var+"_p90"]/10,2)
    print ("Transformed in Celsiuds and rounded")
    Region1i45 = df45C.drop(columns=["realization"])
    print ("pivoted")
    df45f = Region1i45.pivot_table(index=["lat","lon"], columns="time")
    print ("DF pivoted")
    if i== 0: season = "winter" 
    elif i == 1: season = "spring"
    elif i == 2: season = "summer"
    elif i == 3: season = "fall"
    print (season)
    df45f.columns = [season + "_" + year + "rcp45_p"+p
                     for p in ["10", "50", "90"]
                     for year in ["hist_","t2050_", "t2080_"]]
    print("Columns renamed")
    df45list.append(df45f)
    print ("DF added to the list")
    i = i + 1
    

Dataset number:  0 --------------
Percentiles Obteined
Converted to DF
Split in 3 different time periods
Transformed in Celsiuds and rounded
pivoted
DF pivoted
winter
Columns renamed
DF added to the list
Dataset number:  1 --------------
Percentiles Obteined
Converted to DF
Split in 3 different time periods
Transformed in Celsiuds and rounded
pivoted
DF pivoted
spring
Columns renamed
DF added to the list
Dataset number:  2 --------------
Percentiles Obteined
Converted to DF
Split in 3 different time periods
Transformed in Celsiuds and rounded
pivoted
DF pivoted
summer
Columns renamed
DF added to the list
Dataset number:  3 --------------
Percentiles Obteined
Converted to DF
Split in 3 different time periods
Transformed in Celsiuds and rounded
pivoted
DF pivoted
fall
Columns renamed
DF added to the list


In [7]:
#df45list


In [8]:
dfw45 = df45list[0].reset_index()
dfsp45 = df45list[1].reset_index()
dfsu45 = df45list[2].reset_index()
dff45 = df45list[3].reset_index()

mkey = ["lat","lon"]
dfse45 = dfw45.merge(dfsp45, on=mkey).merge(dfsu45, on=mkey).merge(dff45, on=mkey)
dfse45.to_feather("/home/mlopez/EXEC/Processed data to clip with regions/rcp45"+var+"_seasonal.feather")

In [9]:
dfse452 = dfse45.drop(columns=["winter_hist_rcp45_p10","winter_hist_rcp45_p50", "winter_hist_rcp45_p90", 
                               "spring_hist_rcp45_p10", "spring_hist_rcp45_p50", "spring_hist_rcp45_p90", 
                               "summer_hist_rcp45_p10","summer_hist_rcp45_p50", "summer_hist_rcp45_p90", 
                               "fall_hist_rcp45_p10","fall_hist_rcp45_p50", "fall_hist_rcp45_p90"])
dfse452


,lat,lon,winter_t2050_rcp45_p10,winter_t2080_rcp45_p10,winter_t2050_rcp45_p50,winter_t2080_rcp45_p50,winter_t2050_rcp45_p90,winter_t2080_rcp45_p90,spring_t2050_rcp45_p10,spring_t2080_rcp45_p10,...,summer_t2050_rcp45_p50,summer_t2080_rcp45_p50,summer_t2050_rcp45_p90,summer_t2080_rcp45_p90,fall_t2050_rcp45_p10,fall_t2080_rcp45_p10,fall_t2050_rcp45_p50,fall_t2080_rcp45_p50,fall_t2050_rcp45_p90,fall_t2080_rcp45_p90
0,40.041039,-89.045212,1.94,1.32,4.33,3.74,7.50,7.43,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.03,0.01,0.64,0.52
1,40.041039,-88.961884,2.02,1.33,4.53,3.75,7.71,7.62,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.03,0.01,0.65,0.52
2,40.041039,-88.878548,2.06,1.37,4.57,3.81,7.76,7.67,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.03,0.01,0.64,0.52
3,40.041039,-88.795219,2.08,1.39,4.51,3.81,7.68,7.62,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.03,0.01,0.60,0.51
4,40.041039,-88.711891,2.07,1.39,4.45,3.78,7.58,7.60,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.02,0.01,0.55,0.46
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76201,66.623306,-61.629639,7.28,7.50,12.22,12.51,19.04,19.30,5.71,6.59,...,0.33,0.44,2.18,2.70,8.37,7.91,12.90,12.87,19.35,19.44
76202,66.623306,-61.546310,7.56,7.74,12.50,12.83,19.41,19.62,5.99,6.98,...,0.46,0.66,2.50,3.16,9.12,8.92,14.27,13.80,20.27,20.25
76203,66.623306,-61.462978,7.43,7.62,12.33,12.71,19.25,19.42,5.82,6.75,...,0.37,0.54,2.26,2.76,8.49,8.26,13.67,13.26,19.71,19.78
76204,66.623306,-61.379650,7.35,7.58,12.20,12.60,19.21,19.26,5.79,6.69,...,0.37,0.50,2.26,2.65,8.41,8.03,13.25,12.96,19.52,19.58


In [10]:
dfsew = dfse45[["winter_hist_rcp45_p50"]]
dfsesp = dfse45[["spring_hist_rcp45_p50"]]
dfsesu = dfse45[["summer_hist_rcp45_p50"]]
dfsefa = dfse45[["fall_hist_rcp45_p50"]]

# FOR RCP 85

## Subsetting arrays by periods of time with Xclim

In [11]:
dsEns85= ens.create_ensemble(files85)
dsEns85.time.dt.season[0:4]
# Seasonal example 30 y means
listds85 = []
i=0
for s in dsEns85.time.dt.season[0:4]:
    print (i)
    tmp1 = dsEns85.sel(time=(dsEns85.time.dt.year>=1981))
    tmp1 = tmp1.sel(time=(tmp1.time.dt.season==s))
    print(tmp1)
    i = i+1
    listds85.append(tmp1)

0
<xarray.Dataset>
Dimensions:       (lat: 320, lon: 416, realization: 11, time: 120)
Coordinates:
  * lon           (lon) float32 -89.04521 -88.96188 ... -54.54659 -54.46326
  * time          (time) datetime64[ns] 1981-12-01 1982-12-01 ... 2100-12-01
  * lat           (lat) float32 66.62331 66.53998 66.45665 ... 40.12437 40.04104
  * realization   (realization) int64 0 1 2 3 4 5 6 7 8 9 10
Data variables:
    solidprcptot  (realization, time, lat, lon) float32 dask.array<chunksize=(1, 1, 320, 416), meta=np.ndarray>
Attributes:
    Conventions:     CF-1.5
    title:           ACCESS1-3 model output prepared for CMIP5 historical
    history:         CMIP5 compliant file produced from raw ACCESS model outp...
    institution:     CSIRO (Commonwealth Scientific and Industrial Research O...
    source:          ACCESS1-3 2011. Atmosphere: AGCM v1.0 (N96 grid-point, 1...
    redistribution:  Redistribution prohibited. For internal use only.
1
<xarray.Dataset>
Dimensions:       (lat: 320, lo

## Loop to create all 4 dataframes

In [12]:
i = 0
df85list = []
for ds in listds85:
    print ("Dataset number: ", i)
    perc85 = ens.ensemble_percentiles(ds)
    print ("Percentiles Obteined")
    df85 = perc85.to_dataframe()
    df852 = df85.reset_index()
    df853 = df852.loc[df852["realization"] == 0].dropna()
    print ("Converted to DF")
    year_groups = {y:0 for y in range(1980,2011)}
    year_groups.update({y:1 for y in range(2041,2071)})
    year_groups.update({y:2 for y in range(2071,2101)})
    dfp = df853.groupby([df853.time.dt.year.map(year_groups), "lat","lon", "realization"]).mean()
    dfp852 =  dfp.reset_index()
    print ("Split in 3 different time periods")
    df85C = dfp852.copy()
    df85C[var+"_p10"] = round(df85C[var+"_p10"]/10,2)
    df85C[var+"_p50"] = round(df85C[var+"_p50"]/10,2)
    df85C[var+"_p90"] = round(df85C[var+"_p90"]/10,2)
    print ("Transformed in Celsiuds and rounded")
    Region1i85 = df85C.drop(columns=["realization"])
    print ("pivoted")
    df85f = Region1i85.pivot_table(index=["lat","lon"], columns="time")
    print ("DF pivoted")
    if i== 0: season = "winter" 
    elif i == 1: season = "spring"
    elif i == 2: season = "summer"
    elif i == 3: season = "fall"
    print (season)
    df85f.columns = [season + "_" + year + "rcp85_p"+p
                     for p in ["10", "50", "90"]
                     for year in ["hist_","t2050_", "t2080_"]]
    print("Columns renamed")
    df85list.append(df85f)
    print ("DF added to the list")
    i = i + 1
    

Dataset number:  0
Percentiles Obteined
Converted to DF
Split in 3 different time periods
Transformed in Celsiuds and rounded
pivoted
DF pivoted
winter
Columns renamed
DF added to the list
Dataset number:  1
Percentiles Obteined
Converted to DF
Split in 3 different time periods
Transformed in Celsiuds and rounded
pivoted
DF pivoted
spring
Columns renamed
DF added to the list
Dataset number:  2
Percentiles Obteined
Converted to DF
Split in 3 different time periods
Transformed in Celsiuds and rounded
pivoted
DF pivoted
summer
Columns renamed
DF added to the list
Dataset number:  3
Percentiles Obteined
Converted to DF
Split in 3 different time periods
Transformed in Celsiuds and rounded
pivoted
DF pivoted
fall
Columns renamed
DF added to the list


In [13]:
df85list


[                      winter_hist_rcp85_p10  winter_t2050_rcp85_p10  \
 lat       lon                                                         
 40.041039 -89.045212                   2.70                    1.26   
           -88.961884                   2.76                    1.29   
           -88.878548                   2.82                    1.31   
           -88.795219                   2.85                    1.33   
           -88.711891                   2.84                    1.32   
 ...                                     ...                     ...   
 66.623306 -61.629639                   5.92                    7.92   
           -61.546310                   6.12                    8.19   
           -61.462978                   6.03                    8.04   
           -61.379650                   6.00                    7.92   
           -61.296322                   5.54                    7.36   
 
                       winter_t2080_rcp85_p10  winter_hist_rcp

In [14]:
dfw85 = df85list[0].reset_index()
dfsp85 = df85list[1].reset_index()
dfsu85 = df85list[2].reset_index()
dff85 = df85list[3].reset_index()
mkey = ["lat","lon"]
dfse85 = dfw85.merge(dfsp85, on=mkey).merge(dfsu85, on=mkey).merge(dff85, on=mkey)
dfse85

,lat,lon,winter_hist_rcp85_p10,winter_t2050_rcp85_p10,winter_t2080_rcp85_p10,winter_hist_rcp85_p50,winter_t2050_rcp85_p50,winter_t2080_rcp85_p50,winter_hist_rcp85_p90,winter_t2050_rcp85_p90,...,summer_t2080_rcp85_p90,fall_hist_rcp85_p10,fall_t2050_rcp85_p10,fall_t2080_rcp85_p10,fall_hist_rcp85_p50,fall_t2050_rcp85_p50,fall_t2080_rcp85_p50,fall_hist_rcp85_p90,fall_t2050_rcp85_p90,fall_t2080_rcp85_p90
0,40.041039,-89.045212,2.70,1.26,0.51,5.39,3.73,2.22,9.50,7.18,...,0.00,0.00,0.00,0.00,0.22,0.01,0.00,1.20,0.43,0.12
1,40.041039,-88.961884,2.76,1.29,0.51,5.57,3.80,2.27,9.69,7.32,...,0.00,0.00,0.00,0.00,0.23,0.01,0.00,1.22,0.45,0.12
2,40.041039,-88.878548,2.82,1.31,0.51,5.66,3.84,2.32,9.64,7.37,...,0.00,0.00,0.00,0.00,0.24,0.01,0.00,1.18,0.45,0.12
3,40.041039,-88.795219,2.85,1.33,0.53,5.79,3.83,2.32,9.38,7.33,...,0.00,0.00,0.00,0.00,0.24,0.00,0.00,1.21,0.44,0.13
4,40.041039,-88.711891,2.84,1.32,0.50,5.79,3.83,2.29,9.34,7.36,...,0.00,0.00,0.00,0.00,0.22,0.00,0.00,1.19,0.42,0.13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76201,66.623306,-61.629639,5.92,7.92,6.83,10.31,12.62,13.20,16.43,18.95,...,1.58,7.61,7.49,4.33,12.20,12.41,11.07,18.72,19.94,19.36
76202,66.623306,-61.546310,6.12,8.19,7.24,10.53,13.01,13.68,16.69,19.51,...,1.93,8.24,8.13,5.34,12.90,13.16,12.09,19.32,20.82,20.48
76203,66.623306,-61.462978,6.03,8.04,7.05,10.35,12.92,13.51,16.59,19.20,...,1.78,7.75,7.74,4.92,12.60,12.78,11.50,18.84,20.20,19.75
76204,66.623306,-61.379650,6.00,7.92,6.91,10.29,12.76,13.42,16.50,19.00,...,1.57,7.65,7.54,4.70,12.39,12.55,11.32,18.67,19.98,19.40


In [15]:
dfse85.to_feather("/home/mlopez/EXEC/Processed data to clip with regions/rcp85"+var+"_seasonal.feather")

# Merge df45 and 85

# Extract hist

In [16]:
dfse852 = dfse85.drop(columns=["winter_hist_rcp85_p10","winter_hist_rcp85_p50", "winter_hist_rcp85_p90", 
                               "spring_hist_rcp85_p10", "spring_hist_rcp85_p50", "spring_hist_rcp85_p90", 
                               "summer_hist_rcp85_p10","summer_hist_rcp85_p50", "summer_hist_rcp85_p90", 
                               "fall_hist_rcp85_p10","fall_hist_rcp85_p50", "fall_hist_rcp85_p90"])
dfse852


,lat,lon,winter_t2050_rcp85_p10,winter_t2080_rcp85_p10,winter_t2050_rcp85_p50,winter_t2080_rcp85_p50,winter_t2050_rcp85_p90,winter_t2080_rcp85_p90,spring_t2050_rcp85_p10,spring_t2080_rcp85_p10,...,summer_t2050_rcp85_p50,summer_t2080_rcp85_p50,summer_t2050_rcp85_p90,summer_t2080_rcp85_p90,fall_t2050_rcp85_p10,fall_t2080_rcp85_p10,fall_t2050_rcp85_p50,fall_t2080_rcp85_p50,fall_t2050_rcp85_p90,fall_t2080_rcp85_p90
0,40.041039,-89.045212,1.26,0.51,3.73,2.22,7.18,5.58,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.01,0.00,0.43,0.12
1,40.041039,-88.961884,1.29,0.51,3.80,2.27,7.32,5.72,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.01,0.00,0.45,0.12
2,40.041039,-88.878548,1.31,0.51,3.84,2.32,7.37,5.68,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.01,0.00,0.45,0.12
3,40.041039,-88.795219,1.33,0.53,3.83,2.32,7.33,5.66,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.44,0.13
4,40.041039,-88.711891,1.32,0.50,3.83,2.29,7.36,5.67,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.42,0.13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76201,66.623306,-61.629639,7.92,6.83,12.62,13.20,18.95,21.80,5.66,5.91,...,0.37,0.18,2.14,1.58,7.49,4.33,12.41,11.07,19.94,19.36
76202,66.623306,-61.546310,8.19,7.24,13.01,13.68,19.51,22.36,6.06,6.43,...,0.53,0.29,2.61,1.93,8.13,5.34,13.16,12.09,20.82,20.48
76203,66.623306,-61.462978,8.04,7.05,12.92,13.51,19.20,22.10,5.89,6.18,...,0.40,0.22,2.26,1.78,7.74,4.92,12.78,11.50,20.20,19.75
76204,66.623306,-61.379650,7.92,6.91,12.76,13.42,19.00,21.70,5.85,6.17,...,0.39,0.22,2.15,1.57,7.54,4.70,12.55,11.32,19.98,19.40


## Getting historic mean for each season

In [17]:
### Function
def mean_season(season, dfse45, dfse85):
    dfw45 = dfse45[["lat", "lon", season+"_hist_rcp45_p50"]]
    dfw85 = dfse85[["lat", "lon", season+"_hist_rcp85_p50"]]
    dfsew = pd.merge(dfw45, dfw85, on=["lat","lon"])
    dfsew[season+"_hist_p50"] = round((dfsew[season+"_hist_rcp45_p50"]+ dfsew[season+"_hist_rcp85_p50"])/2, 2)
    dftgwh2 = dfsew.reset_index()
    dftgwh3 = dftgwh2[["lat", "lon", season+"_hist_p50"]]
    return (dftgwh3)


### Winter historic mean

In [18]:
dftgwh = mean_season("winter", dfse45, dfse85)
dftgwh

,lat,lon,winter_hist_p50
0,40.041039,-89.045212,5.34
1,40.041039,-88.961884,5.53
2,40.041039,-88.878548,5.61
3,40.041039,-88.795219,5.68
4,40.041039,-88.711891,5.69
...,...,...,...
76201,66.623306,-61.629639,10.58
76202,66.623306,-61.546310,10.82
76203,66.623306,-61.462978,10.66
76204,66.623306,-61.379650,10.56


### Spring historic mean

In [19]:
dftsph = mean_season("spring", dfse45, dfse85)
dftsph

,lat,lon,spring_hist_p50
0,40.041039,-89.045212,0.44
1,40.041039,-88.961884,0.46
2,40.041039,-88.878548,0.48
3,40.041039,-88.795219,0.49
4,40.041039,-88.711891,0.48
...,...,...,...
76201,66.623306,-61.629639,8.94
76202,66.623306,-61.546310,9.27
76203,66.623306,-61.462978,9.07
76204,66.623306,-61.379650,8.93


### Summer historic mean

In [20]:
dftsuh = mean_season("summer", dfse45, dfse85)
dftsuh

,lat,lon,summer_hist_p50
0,40.041039,-89.045212,0.00
1,40.041039,-88.961884,0.00
2,40.041039,-88.878548,0.00
3,40.041039,-88.795219,0.00
4,40.041039,-88.711891,0.00
...,...,...,...
76201,66.623306,-61.629639,0.23
76202,66.623306,-61.546310,0.30
76203,66.623306,-61.462978,0.25
76204,66.623306,-61.379650,0.24


### Fall historic mean

In [21]:
dftfah = mean_season("fall", dfse45, dfse85)
dftfah

,lat,lon,fall_hist_p50
0,40.041039,-89.045212,0.22
1,40.041039,-88.961884,0.22
2,40.041039,-88.878548,0.24
3,40.041039,-88.795219,0.23
4,40.041039,-88.711891,0.21
...,...,...,...
76201,66.623306,-61.629639,12.30
76202,66.623306,-61.546310,12.96
76203,66.623306,-61.462978,12.66
76204,66.623306,-61.379650,12.44


# Merge means with the 45 and 85 DF

In [22]:
dftgseall = dfse852.merge(dfse452, on=mkey).merge(dftfah, on=mkey).merge(dftsuh, on=mkey).merge(dftsph, on=mkey).merge(dftgwh, on=mkey)

In [23]:
dftgseall

,lat,lon,winter_t2050_rcp85_p10,winter_t2080_rcp85_p10,winter_t2050_rcp85_p50,winter_t2080_rcp85_p50,winter_t2050_rcp85_p90,winter_t2080_rcp85_p90,spring_t2050_rcp85_p10,spring_t2080_rcp85_p10,...,fall_t2050_rcp45_p10,fall_t2080_rcp45_p10,fall_t2050_rcp45_p50,fall_t2080_rcp45_p50,fall_t2050_rcp45_p90,fall_t2080_rcp45_p90,fall_hist_p50,summer_hist_p50,spring_hist_p50,winter_hist_p50
0,40.041039,-89.045212,1.26,0.51,3.73,2.22,7.18,5.58,0.00,0.00,...,0.00,0.00,0.03,0.01,0.64,0.52,0.22,0.00,0.44,5.34
1,40.041039,-88.961884,1.29,0.51,3.80,2.27,7.32,5.72,0.00,0.00,...,0.00,0.00,0.03,0.01,0.65,0.52,0.22,0.00,0.46,5.53
2,40.041039,-88.878548,1.31,0.51,3.84,2.32,7.37,5.68,0.00,0.00,...,0.00,0.00,0.03,0.01,0.64,0.52,0.24,0.00,0.48,5.61
3,40.041039,-88.795219,1.33,0.53,3.83,2.32,7.33,5.66,0.00,0.00,...,0.00,0.00,0.03,0.01,0.60,0.51,0.23,0.00,0.49,5.68
4,40.041039,-88.711891,1.32,0.50,3.83,2.29,7.36,5.67,0.00,0.00,...,0.00,0.00,0.02,0.01,0.55,0.46,0.21,0.00,0.48,5.69
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76201,66.623306,-61.629639,7.92,6.83,12.62,13.20,18.95,21.80,5.66,5.91,...,8.37,7.91,12.90,12.87,19.35,19.44,12.30,0.23,8.94,10.58
76202,66.623306,-61.546310,8.19,7.24,13.01,13.68,19.51,22.36,6.06,6.43,...,9.12,8.92,14.27,13.80,20.27,20.25,12.96,0.30,9.27,10.82
76203,66.623306,-61.462978,8.04,7.05,12.92,13.51,19.20,22.10,5.89,6.18,...,8.49,8.26,13.67,13.26,19.71,19.78,12.66,0.25,9.07,10.66
76204,66.623306,-61.379650,7.92,6.91,12.76,13.42,19.00,21.70,5.85,6.17,...,8.41,8.03,13.25,12.96,19.52,19.58,12.44,0.24,8.93,10.56


In [24]:
dftgseall.to_feather("/home/mlopez/EXEC/Processed data to clip with regions/"+var+"_seasonal.feather")

In [25]:
#dftgseall = pd.read_feather("/home/mlopez/EXEC/Processed data to clip with regions/"+var+"_annual.feather")

# Merge complete DF with Polygons for each spatial scale

## Dictionary - regions: column name, short name

In [26]:
short_dict = {"DDE_STF_20K_REG_FOR_VUE_S": ("NM_REG_FOR", "RF"), 
            "DDE_STF_20K_UA_PER_VUE_S": ("PER_NO_UA", "UA"), 
              "DOM_BIO": ("NOM", "DB"), 
              "REG_ECO": ("NOM", "RE"), 
              "SDOM_BIO": ("NOM", "SDB"), 
              "Secteurs_Operations_Regionales": ("D_GENERAL", "SOR"), 
              "SREG_ECO": ("NOM", "SRE"),  
              "territoire_guide": ("TER_GUIDE", "TG") 
              }

for region, (name, short) in short_dict.items():
    print(region, name, short)

DDE_STF_20K_REG_FOR_VUE_S NM_REG_FOR RF
DDE_STF_20K_UA_PER_VUE_S PER_NO_UA UA
DOM_BIO NOM DB
REG_ECO NOM RE
SDOM_BIO NOM SDB
Secteurs_Operations_Regionales D_GENERAL SOR
SREG_ECO NOM SRE
territoire_guide TER_GUIDE TG


In [27]:
def replace_all(text, dic):
    for i, j in dic.items():
        text = text.replace(i, j)
    return text

In [28]:
d = { 'BAS-SAINT-LAURENT':"Bas-Saint-Laurent", 
     'SAGUENAY -LAC-SAINT-JEAN': "Saguenay -Lac-Saint-Jean",
     'CAPITALE-NATIONALE-CHAUDIÈRE-APPALACHES':"Capitale-Nationale-Chaudiere-Appalaches",
     'MAURICIE-CENTRE-DU-QUÉBEC':'Mauricie-Centre-du-Quebec','OUTAOUAIS':'Outaouais', 
     'ABITIBI-TEMISCAMINGUE':'Abitibi-Temiscamingue', 'COTE-NORD':'Cote-Nord',
     'NORD-DU-QUEBEC':'Nord-Du-Quebec', 'GASPESIE-ILES-DE-LA-MADELEINE':'Gaspesie-Iles-De-La-Madeleine', 
     'LANAUDIERE':'Lanaudiere','LAURENTIDES':"Laurentides", 
     "ESTRIE-MONTÉRÉGIE-LAVAL-MONTRÉAL":"Estrie-Monteregie-Laval-Montreal", 
     "é": "e", "É": "E", "à": "a", "è": "e", "Î": "i", "È": "E", "ô" : "o", "Ç":"C", "ç":"c",
       } 

In [29]:
for region, (name, short) in short_dict.items():
    print(region)
    dfpolyshape = from_geofeather('/home/mlopez/EXEC/Grids-polygons-regions/Grid-'+region+'.feather')
    dftp = pd.merge(dftgseall, dfpolyshape, on=["lat","lon"])
    print ("Merged with polygons")
    listTG = []
    for tg in dftp[name].unique().tolist():
        df2 = dftp[dftp[name] == tg]
        print (tg)
        if tg != None:
            listTG.append(df2)
        #print (listTG)
    for df in listTG:
        geometry = df["geometry"]
        crs = {'init': "epsg:4326"}
        gdf = GeoDataFrame(df, crs=crs, geometry=geometry)
        print (gdf[name].iloc[0])
        #Substitute filename accents
        gdf.to_file(replace_all("/home/mlopez/EXEC/GeoJsonMFFP/"+gdf[name].iloc[0], d)+"_"+var+"_seasonal.json", driver="GeoJSON")
    geometry = dftp["geometry"]
    crs = {'init': "epsg:4326"}
    gdf = GeoDataFrame(dftp, crs=crs, geometry=geometry)
    gdf.to_file("/home/mlopez/EXEC/GeoJsonMFFP/"+short+"_"+var+"_seasonal.json", driver="GeoJSON")

  warnings.warn(



DDE_STF_20K_REG_FOR_VUE_S
Merged with polygons
ESTRIE-MONTÉRÉGIE-LAVAL-MONTRÉAL
OUTAOUAIS
LAURENTIDES
MAURICIE-CENTRE-DU-QUÉBEC
LANAUDIERE
CAPITALE-NATIONALE-CHAUDIÈRE-APPALACHES
ABITIBI-TEMISCAMINGUE
BAS-SAINT-LAURENT
GASPESIE-ILES-DE-LA-MADELEINE
SAGUENAY -LAC-SAINT-JEAN
COTE-NORD
NORD-DU-QUEBEC
ESTRIE-MONTÉRÉGIE-LAVAL-MONTRÉAL
OUTAOUAIS
LAURENTIDES
MAURICIE-CENTRE-DU-QUÉBEC
LANAUDIERE
CAPITALE-NATIONALE-CHAUDIÈRE-APPALACHES
ABITIBI-TEMISCAMINGUE
BAS-SAINT-LAURENT
GASPESIE-ILES-DE-LA-MADELEINE
SAGUENAY -LAC-SAINT-JEAN
COTE-NORD
NORD-DU-QUEBEC


  warnings.warn(



DDE_STF_20K_UA_PER_VUE_S
Merged with polygons
05151
07151
07251
06151
03571
06452
06271
07351
07152
08151
06471
04151
03451
08152
07352
03171
07451
04352
04251
01171
03153
08351
02371
03351
08251
04351
01272
08451
02571
11161
09751
11262
08462
08651
02651
08751
08551
02471
11263
08762
02751
02664
09351
08652
02666
08763
09471
08764
02665
08665
02663
08562
08664
08666
02662
08663
09551
02661
09352
05151
07151
07251
06151
03571
06452
06271
07351
07152
08151
06471
04151
03451
08152
07352
03171
07451
04352
04251
01171
03153
08351
02371
03351
08251
04351
01272
08451
02571
11161
09751
11262
08462
08651
02651
08751
08551
02471
11263
08762
02751
02664
09351
08652
02666
08763
09471
08764
02665
08665
02663
08562
08664
08666
02662
08663
09551
02661
09352


  warnings.warn(



...

Massif des monts Groulx


ERROR:fiona._env:/home/mlopez/EXEC/GeoJsonMFFP/Coteaux des lacs Matonipi et Jonquet_solidprcptot_seasonal.json: No such file or directory


Coteaux des lacs Matonipi et Jonquet


  warnings.warn(



territoire_guide
Merged with polygons
1a
2c
3d
2b
2a
3c
3ab
4bc
4de
5ef
4f
4a
5jk
5bcd
5a
4gh
5hi
5g
6cdefg
6hi
6ab
6j
6mn
6kl
6opqr
1a
2c
3d
2b
2a
3c
3ab
4bc
4de
5ef
4f
4a
5jk
5bcd
5a
4gh
5hi
5g
6cdefg
6hi
6ab
6j
6mn
6kl
6opqr
